In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import json

c:\Users\nicol\Documents\ESILV\S9\LLM and GenAI\Projet\python_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
df = pd.read_csv("./data/job_offer/job_descriptions.csv")
df = df.sample(10000)

fields_to_combine = ["Job Id", "Job Title", "Job Description", "skills", "Responsibilities", "Company", "location"]
df["combined_text"] = df[fields_to_combine].astype(str).agg(" ".join, axis=1)

,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Job Title,Role,Job Portal,Job Description,Benefits,skills,Responsibilities,Company,Company Profile,combined_text
328693,1189731731249124,2 to 11 Years,BA,$62K-$95K,Zagreb,Croatia,45.1000,15.2000,Intern,40097,...,Procurement Manager,Procurement Analyst,Idealist,"Analyze procurement data, identify cost-saving...","{'Legal Assistance, Bonuses and Incentive Prog...",Procurement processes Vendor assessment Contra...,Analyze procurement data and trends to support...,Netflix,"{""Sector"":""Entertainment"",""Industry"":""Entertai...",1189731731249124 Procurement Manager Analyze p...
1547009,1850193175036700,4 to 12 Years,M.Com,$62K-$130K,Ouagadougou,Burkina Faso,12.2383,-1.5616,Intern,62597,...,Procurement Manager,Procurement Analyst,SimplyHired,"Analyze procurement data, identify cost-saving...","{'Employee Assistance Programs (EAP), Tuition ...",Procurement processes Vendor assessment Contra...,Analyze procurement data and trends to support...,Halma,"{""Sector"":""Technology and Safety"",""Industry"":""...",1850193175036700 Procurement Manager Analyze p...
1473546,2681485538291292,1 to 12 Years,BCA,$56K-$130K,Saint George's,Grenada,12.1165,-61.6790,Temporary,132579,...,Investment Advisor,Wealth Advisor,Indeed,Wealth Advisors provide financial advice to cl...,"{'Employee Referral Programs, Financial Counse...",Financial planning Investment knowledge Relati...,Provide financial advice and investment strate...,Bharat Electronics Limited,"{""Sector"":""Electronics"",""Industry"":""Electronic...",2681485538291292 Investment Advisor Wealth Adv...
241100,172924570544942,2 to 10 Years,BA,$64K-$125K,Santo Domingo,Dominican Republic,18.7357,-70.1627,Full-Time,68675,...,Administrative Assistant,Executive Assistant,Dice,Support high-level executives by managing sche...,"{'Legal Assistance, Bonuses and Incentive Prog...",Calendar management Travel coordination Meetin...,Provide high-level administrative support to e...,Metro AG,"{""Sector"":""Retail"",""Industry"":""Retail"",""City"":...",172924570544942 Administrative Assistant Suppo...
607894,2050462086045339,1 to 12 Years,MCA,$56K-$93K,Honiara,Solomon Islands,-9.6457,160.1562,Full-Time,41353,...,Back-End Developer,Server Developer,Internships.com,A Server Developer specializes in developing a...,"{'Health Insurance, Retirement Plans, Flexible...","Server-side programming languages (e.g., Java,...",Develop and maintain server-side logic and dat...,ICICI Bank,"{""Sector"":""Banking"",""Industry"":""Banking and Fi...",2050462086045339 Back-End Developer A Server D...


In [3]:
model = SentenceTransformer("all-MiniLM-L6-v2")
job_embeddings = model.encode(df["combined_text"].tolist(), convert_to_numpy=True, show_progress_bar=True, normalize_embeddings=True)

faiss_index = faiss.IndexFlatIP(job_embeddings.shape[1])
faiss_index.add(job_embeddings)
faiss.write_index(faiss_index, "./data/jobs_index.faiss")

job_id_mapping = {i: jid for i, jid in enumerate(df["Job Id"].tolist())}
with open("./data/jobs_index_mapping.json", "w", encoding="utf-8") as f:
    json.dump(job_id_mapping, f, indent=4)

Batches: 100%|██████████| 313/313 [03:48<00:00,  1.37it/s]
